<a href="https://colab.research.google.com/github/JulianCarax01/Fondamenti-web-app/blob/main/Dijkstra_LISTA_ADIACENZA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
int main() {
    int n = 2048;         // Numero di nodi
    float minWeight = 1.0f;    // Peso minimo degli archi
    float maxWeight = 2000.0f; // Peso massimo degli archi

    // 1) Generazione del grafo come lista di adiacenza (seriale)
    std::cout << "Generazione della lista di adiacenza...\n";
    auto startTime = std::chrono::high_resolution_clock::now();

    std::vector<std::vector<Edge>> adjacencyList;
    generateAdjListSerial(adjacencyList, n, minWeight, maxWeight);
%%writefile dijkstra.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <climits>
#include <cfloat> // Per FLT_MAX
#include <cuda.h>
#include <curand_kernel.h>
#include <chrono>
#include <algorithm>

//////////////////////////////////////////////////////////
// Struttura per memorizzare un arco in modo sparso
//////////////////////////////////////////////////////////
struct Edge {
    int to;
    float weight;
};

//////////////////////////////////////////////////////////
// Generazione grafo seriale in forma di lista, su CPU
// Genera in modo deterministico un grafo sparso:
// - 40% di probabilità di creare un arco
// - Simmetrico
// - Con un peso random compreso tra minWeight e maxWeight
//////////////////////////////////////////////////////////
void generateAdjListSerial(std::vector<std::vector<Edge>>& adjacencyList,
                           int n, float minWeight, float maxWeight)
{
    std::srand(std::time(nullptr));

    adjacencyList.resize(n);

    for (int row = 0; row < n; ++row) {
        for (int col = row + 1; col < n; ++col) {
            int randomValue = std::rand() % 100;
            if (randomValue < 40) {
                // Crea un arco
                float weight = static_cast<float>(std::rand()) / RAND_MAX
                               * (maxWeight - minWeight) + minWeight;

                // Aggiunge l'arco row->col
                adjacencyList[row].push_back({col, weight});
                // Aggiunge l'arco col->row
                adjacencyList[col].push_back({row, weight});
            }
        }
    }
}

//////////////////////////////////////////////////////////
// Dijkstra Serial su lista di adiacenza, su CPU
// allDistances conterra' le distanze minime dal nodo i a tutti gli altri,
// memorizzate in allDistances[i*n + j].
// adjacencyList[node] e' la lista di tutti i (to, weight).
//////////////////////////////////////////////////////////
void dijkstraSerialList(const std::vector<std::vector<Edge>>& adjacencyList,
                        int n, float* allDistances)
{
    for (int start = 0; start < n; ++start) {
        // Array di visited
        std::vector<int> visited(n, 0);
        float* distances = &allDistances[start * n];
        for (int i = 0; i < n; ++i) {
            distances[i] = (i == start) ? 0.0f : FLT_MAX;
        }

        // Dijkstra classico
        for (int step = 0; step < n; ++step) {
            float minDist = FLT_MAX;
            int currentNode = -1;
            for (int j = 0; j < n; ++j) {
                if (!visited[j] && distances[j] < minDist) {
                    minDist = distances[j];
                    currentNode = j;
                }
            }
            if (currentNode == -1) break;
            visited[currentNode] = 1;

            // Rilassa i vicini di currentNode
            for (auto& edge : adjacencyList[currentNode]) {
                int neighbor = edge.to;
                float w = edge.weight;
                if (!visited[neighbor] &&
                    distances[currentNode] + w < distances[neighbor]) {
                    distances[neighbor] = distances[currentNode] + w;
                }
            }
        }
    }
}

//////////////////////////////////////////////////////////
// Struttura per salvare la forma sparsa su GPU
//////////////////////////////////////////////////////////
struct GpuEdge {
    int to;
    float weight;
};

//////////////////////////////////////////////////////////
// buildSparseArrays:
// Data adjacencyList[node], costruiamo i due array:
//   edges[] e nodeOffset[], da passare a GPU
//////////////////////////////////////////////////////////
void buildSparseArrays(const std::vector<std::vector<Edge>>& adjacencyList,
                       std::vector<GpuEdge>& edges,
                       std::vector<int>& nodeOffset)
{
    int n = (int) adjacencyList.size();
    int totalEdges = 0;
    for (int i = 0; i < n; i++) {
        totalEdges += (int) adjacencyList[i].size();
    }
    edges.resize(totalEdges);
    nodeOffset.resize(n + 1, 0);

    for (int i = 0; i < n; i++) {
        nodeOffset[i+1] = nodeOffset[i] + (int) adjacencyList[i].size();
    }

    for (int i = 0; i < n; i++) {
        int startIdx = nodeOffset[i];
        for (int k = 0; k < (int) adjacencyList[i].size(); k++) {
            edges[startIdx + k].to     = adjacencyList[i][k].to;
            edges[startIdx + k].weight = adjacencyList[i][k].weight;
        }
    }
}

//////////////////////////////////////////////////////////
// Kernel dijkstraListKernel:
// 1 thread = 1 nodo sorgente
//////////////////////////////////////////////////////////
__global__ void dijkstraListKernel(const GpuEdge* edges,
                                   const int* nodeOffset,
                                   float* allDistances,
                                   int* allVisited,
                                   int n)
{
    int s = blockIdx.x * blockDim.x + threadIdx.x;
    if (s >= n) return;

    float* distances = &allDistances[s * n];
    int* visited     = &allVisited[s * n];

    // Inizializza
    for (int i = 0; i < n; i++) {
        distances[i] = (i == s) ? 0.0f : FLT_MAX;
        visited[i]   = 0;
    }

    // Dijkstra classico
    for (int step = 0; step < n; step++) {
        float minDist = FLT_MAX;
        int   curr    = -1;

        for (int i = 0; i < n; i++) {
            if (!visited[i] && distances[i] < minDist) {
                minDist = distances[i];
                curr    = i;
            }
        }
        if (curr == -1) break;
        visited[curr] = 1;

        // Rilassa i vicini di curr
        int startIdx = nodeOffset[curr];
        int endIdx   = nodeOffset[curr+1];
        for (int idx = startIdx; idx < endIdx; idx++) {
            int neighbor = edges[idx].to;
            float w      = edges[idx].weight;
            if (!visited[neighbor] && distances[curr] + w < distances[neighbor]) {
                distances[neighbor] = distances[curr] + w;
            }
        }
    }
}

//////////////////////////////////////////////////////////
// DijkstraListCUDA:
// Costruisce arrays sparsi, li copia in GPU e lancia kernel.
//////////////////////////////////////////////////////////
void dijkstraListCUDA(const std::vector<std::vector<Edge>>& adjacencyList,
                       int n,
                       float* distances)
{
    // 1. Prepara array edges e nodeOffset su CPU
    std::vector<GpuEdge> cpuEdges;
    std::vector<int> cpuNodeOffset;
    buildSparseArrays(adjacencyList, cpuEdges, cpuNodeOffset);

    // 2. Alloca su GPU
    GpuEdge* d_edges;
    int* d_nodeOffset;
    float* d_distances;
    int* d_visited;

    int totalEdges = (int) cpuEdges.size();

    cudaMalloc(&d_edges,      totalEdges * sizeof(GpuEdge));
    cudaMalloc(&d_nodeOffset, (n + 1) * sizeof(int));
    cudaMalloc(&d_distances,  n * n * sizeof(float));
    cudaMalloc(&d_visited,    n * n * sizeof(int));

    // Copia arrays su GPU
    cudaMemcpy(d_edges, cpuEdges.data(), totalEdges * sizeof(GpuEdge),
               cudaMemcpyHostToDevice);
    cudaMemcpy(d_nodeOffset, cpuNodeOffset.data(), (n+1)*sizeof(int),
               cudaMemcpyHostToDevice);

    // 3. Lancia kernel
    int blockSize = 128;
    int gridSize  = (n + blockSize - 1) / blockSize;

    dijkstraListKernel<<<gridSize, blockSize>>>(
        d_edges,
        d_nodeOffset,
        d_distances,
        d_visited,
        n
    );
    cudaDeviceSynchronize();

    // 4. Copia risultato su CPU
    cudaMemcpy(distances, d_distances, n * n * sizeof(float),
               cudaMemcpyDeviceToHost);

    // 5. Dealloca su GPU
    cudaFree(d_edges);
    cudaFree(d_nodeOffset);
    cudaFree(d_distances);
    cudaFree(d_visited);
}

//////////////////////////////////////////////////////////
int main() {
    int n = 4096;         // Numero di nodi
    float minWeight = 1.0f;    // Peso minimo degli archi
    float maxWeight = 2000.0f; // Peso massimo degli archi

    // 1 Generazione del grafo come lista di adiacenza (seriale)
    std::cout << "Generazione della lista di adiacenza...\n";
    auto startTime = std::chrono::high_resolution_clock::now();

    std::vector<std::vector<Edge>> adjacencyList;
    generateAdjListSerial(adjacencyList, n, minWeight, maxWeight);

    auto endTime = std::chrono::high_resolution_clock::now();
    long serialGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(
        endTime - startTime
    ).count();
    std::cout << "Tempo di generazione lista (seriale): "
              << serialGraphTime << " µs\n";

    // 2 Allochiamo gli array per salvare i risultati di Dijkstra
    std::vector<float> serialDistances(n * n, FLT_MAX);
    std::vector<float> cudaDistances(n * n, FLT_MAX);

    // 3 Dijkstra seriale sulla lista di adiacenza
    std::cout << "\nEsecuzione Dijkstra Serial sulla lista di adiacenza...\n";
    startTime = std::chrono::high_resolution_clock::now();

    dijkstraSerialList(adjacencyList, n, serialDistances.data());

    endTime = std::chrono::high_resolution_clock::now();
    long serialTime = std::chrono::duration_cast<std::chrono::microseconds>(
        endTime - startTime
    ).count();

    // 4 Dijkstra in CUDA (1 thread = 1 nodo sorgente)
    std::cout << "\nEsecuzione Dijkstra CUDA sulla lista di adiacenza...\n";
    startTime = std::chrono::high_resolution_clock::now();

    dijkstraListCUDA(adjacencyList, n, cudaDistances.data());

    endTime = std::chrono::high_resolution_clock::now();
    long cudaTime = std::chrono::duration_cast<std::chrono::microseconds>(
        endTime - startTime
    ).count();

    // 5 Confronto a campione
    std::cout << "\n=== Confronto Distanze (Serial vs. CUDA) ===\n";
    std::cout << "Nodo 230, prime 32 distanze:\n";
    for (int j = 0; j < 32; j++) {
        float cDist = cudaDistances[230 * n + j];
        float sDist = serialDistances[230 * n + j];
        std::cout << "  j=" << j << ": CUDA=";
        if (cDist == FLT_MAX) std::cout << "∞";
        else std::cout << cDist;

        std::cout << ", SERIAL=";
        if (sDist == FLT_MAX) std::cout << "∞";
        else std::cout << sDist;

        std::cout << "\n";
    }

    // 6 Stampa i tempi
    std::cout << "\n=== Tempi di esecuzione ===\n";
    std::cout << "Dijkstra Serial: " << serialTime << " µs\n";
    std::cout << "Dijkstra CUDA:   " << cudaTime   << " µs\n";

    return 0;
}

    auto endTime = std::chrono::high_resolution_clock::now();
    long serialGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(
        endTime - startTime
    ).count();
    std::cout << "Tempo di generazione lista (seriale): "
              << serialGraphTime << " µs\n";

    // 2) Allochiamo gli array per salvare i risultati di Dijkstra
    std::vector<float> serialDistances(n * n, FLT_MAX);
    std::vector<float> cudaDistances(n * n, FLT_MAX);

    // 3) Dijkstra seriale sulla lista di adiacenza
    std::cout << "\nEsecuzione Dijkstra Serial sulla lista di adiacenza...\n";
    startTime = std::chrono::high_resolution_clock::now();

    dijkstraSerialList(adjacencyList, n, serialDistances.data());

    endTime = std::chrono::high_resolution_clock::now();
    long serialTime = std::chrono::duration_cast<std::chrono::microseconds>(
        endTime - startTime
    ).count();

    // 4) Dijkstra in CUDA (1 thread = 1 nodo sorgente)
    std::cout << "\nEsecuzione Dijkstra CUDA sulla lista di adiacenza...\n";
    startTime = std::chrono::high_resolution_clock::now();

    dijkstraListCUDA(adjacencyList, n, cudaDistances.data());

    endTime = std::chrono::high_resolution_clock::now();
    long cudaTime = std::chrono::duration_cast<std::chrono::microseconds>(
        endTime - startTime
    ).count();

    // 5) Confronto a campione
    std::cout << "\n=== Confronto Distanze (Serial vs. CUDA) ===\n";
    std::cout << "Nodo 230, prime 32 distanze:\n";
    for (int j = 0; j < 32; j++) {
        float cDist = cudaDistances[230 * n + j];
        float sDist = serialDistances[230 * n + j];
        std::cout << "  j=" << j << ": CUDA=";
        if (cDist == FLT_MAX) std::cout << "∞";
        else std::cout << cDist;

        std::cout << ", SERIAL=";
        if (sDist == FLT_MAX) std::cout << "∞";
        else std::cout << sDist;

        std::cout << "\n";
    }

    // 6) Stampa i tempi
    std::cout << "\n=== Tempi di esecuzione ===\n";
    std::cout << "Dijkstra Serial: " << serialTime << " µs\n";
    std::cout << "Dijkstra CUDA:   " << cudaTime   << " µs\n";

    return 0;
}


Overwriting debug.cu


In [ ]:
!nvcc -G -g debug.cu -o debug.o
!./debug.o

Generazione del grafo...
Tempo di generazione del grafo: 217 µs

=== Matrice del grafo iniziale ===
0 ∞ ∞ ∞ ∞ ∞ ∞ ∞ ∞ 486.336 ∞ 1397.82 ∞ ∞ 1248.47 542.702 ∞ ∞ ∞ ∞ 1053.45 ∞ ∞ 1544.8 586.13 1756.56 1057.58 1425.58 ∞ ∞ ∞ ∞ ∞ 1247.51 1925.29 ∞ ∞ ∞ ∞ 1441.58 1744.45 ∞ ∞ 1798.04 ∞ ∞ 944.811 ∞ 1934.18 830.611 ∞ ∞ ∞ 1869.32 73.0076 ∞ ∞ ∞ ∞ ∞ ∞ ∞ ∞ ∞ ∞ 1047.71 ∞ ∞ ∞ ∞ 1887.11 556.398 ∞ ∞ ∞ 466.862 1963.66 ∞ ∞ ∞ ∞ 355.383 ∞ 1662.12 ∞ ∞ ∞ ∞ 847.922 ∞ ∞ ∞ ∞ ∞ 1990.17 1799.02 ∞ ∞ ∞ 641.113 ∞ ∞ ∞ ∞ ∞ 
∞ 0 1558.13 1954.46 ∞ ∞ ∞ ∞ ∞ ∞ ∞ 38.882 892.075 ∞ 135.698 ∞ ∞ ∞ 1053.33 ∞ 1693.44 ∞ 725.229 ∞ ∞ ∞ 225.949 1617.55 ∞ 69.4174 ∞ 317.981 1560.58 ∞ ∞ ∞ 1990.51 ∞ 1924.23 1974.99 ∞ ∞ ∞ ∞ 393.658 ∞ 612.364 ∞ ∞ ∞ ∞ ∞ 350.377 ∞ ∞ ∞ 610.946 306.017 ∞ ∞ ∞ 1865.28 ∞ 1840.27 ∞ ∞ ∞ ∞ ∞ ∞ ∞ 513.725 ∞ 1298.26 1163.04 ∞ ∞ ∞ 1681.8 ∞ 581.828 797.697 1698.25 1389.73 ∞ ∞ ∞ ∞ ∞ ∞ ∞ ∞ 1867.63 1152.79 ∞ 451.05 65.1506 ∞ ∞ ∞ ∞ ∞ ∞ ∞ ∞ 
∞ 1558.13 0 434.172 910.339 ∞ 828.355 339.05 ∞ ∞ 1980.72 ∞ 1848.36 ∞ ∞ ∞ ∞ ∞ 431.414 ∞ 

In [ ]:
!ncu --set full -o test_rep3 ./debug.o

==PROF== Connected to process 2260 (/content/debug.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 31 passes
==PROF== Profiling "initialValueKernel" - 1: 0%....50%....100% - 31 passes
==PROF== Profiling "findMinReductionCUDA" - 2: 0%....50%....100% - 31 passes
==PROF== Profiling "cudaNodeRelax" - 3: 0%....50%....100% - 31 passes
==PROF== Profiling "findMinReductionCUDA" - 4: 0%....50%....100% - 31 passes
==PROF== Profiling "cudaNodeRelax" - 5: 0%....50%....100% - 31 passes
==PROF== Profiling "findMinReductionCUDA" - 6: 0%....50%....100% - 31 passes
==PROF== Profiling "cudaNodeRelax" - 7: 0%....50%....100% - 31 passes
==PROF== Profiling "findMinReductionCUDA" - 8: 0%....50%....100% - 31 passes
==PROF== Profiling "cudaNodeRelax" - 9: 0%....50%....100% - 31 passes
==PROF== Profiling "findMinReductionCUDA" - 10: 0%....50%....100% - 31 passes
==PROF== Profiling "cudaNodeRelax" - 11: 0%....50%....100% - 31 passes
==PROF== Received signal
==PROF== Trying to shutdown targe